# DIY Covid-19 Dashboard

The following data aims to compare the number of new cases of COVID-19 with the number of new tests carried out. The data should provide a better picture of the rate of infection and should also provide detail regarding the porportion of people testing positive for COVID-19. The query was made all across England and has been presented on a timescale of about a year.

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [3]:

filters = [
    'areaType=nation',
    'areaName=England'
    
]
structure = {
    "date": "date",
    "newCases":"newCasesByPublishDate",
    "newTestsCarriedOut":"newTestsByPublishDate"
}
api = Cov19API(filters=filters, structure=structure)
testData=api.get_json()
with open("testData.json", "wt") as OUTF:
    json.dump(testData, OUTF)
jsondata=testData['data']
jsondata[0]['newTestsCarriedOut']= 0 #this is to ensure that all data values are integers and not strings


In [4]:


def wrangle_data(rawdata):
    plt.rcParams['figure.dpi'] = 100
    with open("testData.json", "rt") as INFILE:
        data=json.load(INFILE)
    data['data'][0]['newTestsCarriedOut']= 0
    dates=[dictionary['date'] for dictionary in rawdata ]
    dates.sort()
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    testDatadf=pd.DataFrame(index=index,columns=['newCases', 'newTestsCarriedOut'])
    for entry in rawdata:
        date=parse_date(entry['date'])
        for column in ['newCases', 'newTestsCarriedOut']:

            if pd.isna(testDatadf.loc[date, column]): 
 
                value= float(entry[column]) if entry[column]!=None else 0.0

                testDatadf.loc[date, column]=value
    df = testDatadf
    return df

dataframe=wrangle_data(jsondata) 

In [5]:

def access_api():
    filters = [
    'areaType=nation',
    'areaName=England']
    structure = {
        "date": "date",
        "newCases":"newCasesByPublishDate",
        "newTestsCarriedOut":"newTestsByPublishDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    testData=api.get_json()
    with open("testData.json", "wt") as OUTF:
        json.dump(testData, OUTF)
    jsondata=testData['data']
    jsondata[0]['newTestsCarriedOut']= 0
    return jsondata 


# Refresh button
The following button will allow users to refresh the api data effectively updating the data as they see fit

In [6]:

def api_button_callback(button):
    apidata=access_api()

    global df
    df=wrangle_data(apidata)

    refresh_graph()

    apibutton.icon="check"


    
apibutton=wdg.Button(
    description='Refresh', 
    disabled=False,
    button_style='info', 
    tooltip="Keep calm and carry on",
    icon='refresh'
)


apibutton.on_click(api_button_callback) 
display(apibutton)



Button(button_style='info', description='Refresh', icon='refresh', style=ButtonStyle(), tooltip='Keep calm and…

## Graphs and Analysis

The following graph presents the query and has an interactive widget wherby users can switch between graph data.

In [7]:
cols=wdg.SelectMultiple(
    options=['newCases', 'newTestsCarriedOut'], # options available
    value=['newCases', 'newTestsCarriedOut'], # initial value
    rows=2, 
    description='data',
    disabled=False
  
)
def plot(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        dataframe.plot(y=list(graphcolumns)) 
    else:
        print("Click to select data for graph")

    
   
output=wdg.interactive_output(plot, {'graphcolumns': cols})

display(cols, output)


def refresh_graph():
    current=cols.value
    if current==cols.options[0]:
        other=(cols.options[1],)
    else:
        other=(cols.options[0],)
    cols.value=other # forces the redraw
    cols.value=current # now we can change it back
    
    

SelectMultiple(description='data', index=(0, 1), options=('newCases', 'newTestsCarriedOut'), rows=2, value=('n…

Output()